In [1]:
!pip install ensemble_boxes

In [8]:
import pandas as pd
from ensemble_boxes import *
import numpy as np
from pycocotools.coco import COCO

In [51]:
# ensemble csv files
submission_files = ['./atss_dyhead.csv', './yolov5_061.csv', './cascade_rcnn.csv'] # submission lists
submission_df = [pd.read_csv(file) for file in submission_files]

ensemble_submission = './atss_dyhead+yolov5_061.csv'

In [52]:
image_ids = submission_df[0]['image_id'].tolist()

In [53]:
# ensemble 할 file의 image 정보를 불러오기 위한 json
annotation = '/opt/ml/detection/dataset/test.json'
coco = COCO(annotation)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [49]:
prediction_strings = []
file_names = []
# ensemble 시 설정할 iou threshold 이 부분을 바꿔가며 대회 metric에 알맞게 적용해봐요!
iou_thr = 0.6

# 각 image id 별로 submission file에서 box좌표 추출
for i, image_id in enumerate(image_ids):
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []
    image_info = coco.loadImgs(i)[0]
#     각 submission file 별로 prediction box좌표 불러오기
    for df in submission_df:
        if image_id in list(df['image_id']):
            predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
            predict_list = str(predict_string).split()
            
            if len(predict_list)==0 or len(predict_list)==1:
                continue
                
            predict_list = np.reshape(predict_list, (-1, 6))
            box_list = []
            
            for box in predict_list[:, 2:6].tolist():
                box[0] = float(box[0]) / image_info['width']
                box[1] = float(box[1]) / image_info['height']
                box[2] = float(box[2]) / image_info['width']
                box[3] = float(box[3]) / image_info['height']
                box_list.append(box)
                
            boxes_list.append(box_list)
            scores_list.append(list(map(float, predict_list[:, 1].tolist())))
            labels_list.append(list(map(int, predict_list[:, 0].tolist())))
    
#     예측 box가 있다면 이를 ensemble 수행
    if len(boxes_list):
        weights = [1, 1, 0.5]
        boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr)
        for box, score, label in zip(boxes, scores, labels):
            prediction_string += str(int(label)) + ' ' + str(score) + ' ' + str(box[0] * image_info['width']) + ' ' + str(box[1] * image_info['height']) + ' ' + str(box[2] * image_info['width']) + ' ' + str(box[3] * image_info['height']) + ' '
    
    prediction_strings.append(prediction_string)
    file_names.append(image_id)

In [50]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(ensemble_submission, index=False)

submission.head()

PredictionString       image_id
0  7 0.902607798576355 603.9014892578125 518.3752...  test/0000.jpg
1  5 0.8934339284896851 133.79405212402344 0.0 49...  test/0001.jpg
2  0 0.774541974067688 879.8282470703125 463.2749...  test/0002.jpg
3  9 0.5973142385482788 150.15196228027344 252.36...  test/0003.jpg
4  1 0.7617464065551758 187.9369659423828 259.493...  test/0004.jpg